In [1]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from tensorflow.keras.layers import (
    Input, Embedding, Dense, Add,
    LayerNormalization, MultiHeadAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# ---------------- CONFIG ----------------
DATA_PATH = r"C:\Users\koush\Downloads\english_to_telugu_data.csv"        # English, Telugu columns
VOCAB_SIZE = 16000
EMB_DIM = 256
NUM_HEADS = 4
FF_DIM = 512
ENC_LAYERS = 6
DEC_LAYERS = 6
BATCH_SIZE = 50
EPOCHS = 2
MAX_LEN = 40
#use_rows = 50000

In [3]:
df = pd.read_csv(DATA_PATH)
# df = df.iloc[:use_rows, :]

In [4]:
df.shape

(1499448, 2)

In [5]:

df = df[["english", "telugu"]].dropna()

def clean(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\u0C00-\u0C7F\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["english"] = df["english"].apply(clean)
df["telugu"]  = df["telugu"].apply(clean)


In [6]:
df.shape

(1499448, 2)

In [7]:
with open("eng.txt", "w", encoding="utf-8") as f:
    for i in df["english"]:
        f.write(i + "\n")

spm.SentencePieceTrainer.train(
    input="eng.txt",
    model_prefix="spm_eng",
    vocab_size=VOCAB_SIZE,
    model_type="bpe",
    pad_id=0, bos_id=1, eos_id=2, unk_id=3
)


In [8]:
with open("tel.txt", "w", encoding="utf-8") as f:
    for s in df["telugu"]:
        f.write(s + "\n")

spm.SentencePieceTrainer.train(
    input="tel.txt",
    model_prefix="spm_tel",
    vocab_size=VOCAB_SIZE,
    model_type="bpe",
    pad_id=0, bos_id=1, eos_id=2, unk_id=3
)


In [ ]:
sp_eng = spm.SentencePieceProcessor(model_file="spm_eng.model")
sp_tel = spm.SentencePieceProcessor(model_file="spm_tel.model")

In [ ]:
def encode_eng(text):
    return sp_eng.encode(text, out_type=int)

def encode_tel(text):
    return [1] + sp_tel.encode(text, out_type=int) + [2]   # <bos> <eos>


In [ ]:
eng_seq = [encode_eng(s) for s in df["english"]]
tel_seq = [encode_tel(s) for s in df["telugu"]]

In [ ]:
eng_pad = pad_sequences(eng_seq, maxlen=MAX_LEN, padding="post")
tel_pad = pad_sequences(tel_seq, maxlen=MAX_LEN, padding="post")

In [ ]:
decoder_input  = tel_pad[:, :-1]
decoder_target = tel_pad[:, 1:]

In [ ]:
decoder_input = pad_sequences(
    decoder_input, maxlen=MAX_LEN, padding="post"
)

decoder_target = pad_sequences(
    decoder_target, maxlen=MAX_LEN, padding="post"
)


# This is positional_encoding

In [ ]:
def positional_encoding(length, depth):
    depth = depth / 2
    positions = np.arange(length)[:, None]
    depths = np.arange(depth)[None, :] / depth
    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates
    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1
    )
    return tf.cast(pos_encoding[None, ...], tf.float32)

pos_enc = positional_encoding(MAX_LEN, EMB_DIM)

# This ia a Transformer model with 6 Encoders and 6 Decoders

In [ ]:
enc_in = Input(shape=(MAX_LEN,))
dec_in = Input(shape=(MAX_LEN,))
enc_emb = Embedding(VOCAB_SIZE, EMB_DIM, mask_zero=True)(enc_in)
enc_x = Add()([enc_emb, pos_enc])

for _ in range(ENC_LAYERS):
    attn = MultiHeadAttention(NUM_HEADS, EMB_DIM // NUM_HEADS)(
        enc_x, enc_x
    )
    enc_x = LayerNormalization()(Add()([enc_x, attn]))

    ff = Dense(FF_DIM, activation="relu")(enc_x)
    ff = Dense(EMB_DIM)(ff)
    enc_x = LayerNormalization()(Add()([enc_x, ff]))

encoder_out = enc_x
look_ahead_mask = tf.linalg.band_part(
    tf.ones((MAX_LEN, MAX_LEN)), -1, 0
)

dec_emb = Embedding(VOCAB_SIZE, EMB_DIM)(dec_in)
dec_x = Add()([dec_emb, pos_enc])

for _ in range(DEC_LAYERS):
    self_attn = MultiHeadAttention(
        NUM_HEADS, EMB_DIM // NUM_HEADS
    )(dec_x, dec_x, attention_mask=look_ahead_mask)

    dec_x = LayerNormalization()(Add()([dec_x, self_attn]))

    cross_attn = MultiHeadAttention(
        NUM_HEADS, EMB_DIM // NUM_HEADS
    )(dec_x, encoder_out)

    dec_x = LayerNormalization()(Add()([dec_x, cross_attn]))

    ff = Dense(FF_DIM, activation="relu")(dec_x)
    ff = Dense(EMB_DIM)(ff)
    dec_x = LayerNormalization()(Add()([dec_x, ff]))


out = Dense(VOCAB_SIZE, activation="softmax")(dec_x)

model = Model([enc_in, dec_in], out)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 40, 256)   │  4,096,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (1, 40, 256)      │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 40, 256)      │    263,168 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (1, 40, 256)      │          0 │ add[0][0],        │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (1, 40, 256)      │        512 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (1, 40, 512)      │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (1, 40, 256)      │    131,328 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (1, 40, 256)      │          0 │ layer_normalizat… │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 40, 256)      │        512 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 40, 256)      │    263,168 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (1, 40, 256)      │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 40, 256)      │        512 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (1, 40, 512)      │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (1, 40, 256)      │    131,328 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (1, 40, 256)      │          0 │ layer_normalizat… │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 40, 256)      │        512 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 40, 256)      │    263,168 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 20,211,328 (77.10 MB)

 Trainable params: 20,211,328 (77.10 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.load_weights("/content/translator_4lks_15.weights.h5")


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 514 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
history = model.fit(
    [eng_pad, decoder_input],
    decoder_target,
    batch_size=BATCH_SIZE,
    epochs=2,
    validation_split=0.2
)

Epoch 1/2
2304/9600 ━━━━━━━━━━━━━━━━━━━━ 10:51 89ms/step - accuracy: 0.9093 - loss: 0.4429

In [ ]:
import keras
model = keras.models.load_model("/content/translator_4lks_15.keras")

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 40, 256)   │  4,096,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (1, 40, 256)      │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 40, 256)      │    263,168 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (1, 40, 256)      │          0 │ add[0][0],        │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (1, 40, 256)      │        512 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (1, 40, 512)      │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (1, 40, 256)      │    131,328 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (1, 40, 256)      │          0 │ layer_normalizat… │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 40, 256)      │        512 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 40, 256)      │    263,168 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (1, 40, 256)      │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 40, 256)      │        512 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (1, 40, 512)      │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (1, 40, 256)      │    131,328 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (1, 40, 256)      │          0 │ layer_normalizat… │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 40, 256)      │        512 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 40, 256)      │    263,168 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 60,633,986 (231.30 MB)

 Trainable params: 20,211,328 (77.10 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,422,658 (154.20 MB)

In [ ]:
import keras
keras.models.save_model(model, "/content/translator_6lks_29.keras")

In [ ]:
model.save_weights("/content/translator_6lks_29.weights.h5")